In [1]:
from html import escape

In [3]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(k, v)
            for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [4]:
print(html_str("""this is
a multi line string
with special characters: 10 < 100"""))

this is<br/>
a multi line string<br/>
with special characters: 10 &lt; 100


In [5]:
print(html_int(255))

255(<i>0xff</i>)


In [6]:
print(html_escape(3+10j))

(3+10j)


In [7]:
from decimal import Decimal

In [8]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    
    elif isinstance(arg, str):
        return html_str(arg)
    
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    
    elif isinstance(arg, dict):
        return html_dict(arg)
    
    else:
        return html_escape(arg)

In [9]:
htmlize("""Python
rocks!
""")

'Python<br/>\nrocks!<br/>\n'

In [11]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [13]:
# problem with this -> our nested structures don't get htmlified correctly
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python
rocks! 0 &lt; 1
</li>
<li>(10, 20, 30)</li>
<li>100</li>
</ul>


In [18]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)
    

def html_escape(arg):
    return escape(str(arg))


def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))


def html_real(a):
    return '{0:.2f}'.format(round(a, 2))


def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')


def html_list(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), htmlize(v))
            for k, v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
    

In [15]:
# this approach works, but the problem is every time we add a new type, we need to add code
# in multiple places
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python<br/>
rocks! 0 &lt; 1<br/>
</li>
<li><ul>
<li>10(<i>0xa</i>)</li>
<li>20(<i>0x14</i>)</li>
<li>30(<i>0x1e</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


In [19]:
# a better approach would be to maintain a dictionary of types instead of if/elif/else

def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list, # we should be using abstract base classes here -> will cover later
        tuple: html_list,
        set: html_set,
        dict: html_dict
    }
    
    fn = registry.get(type(arg), registry[object])
    
    return fn(arg)

In [20]:
htmlize(100)

'100(<i>0x64</i>)'

In [21]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [22]:
# now, let's begin to turn it into a dispatcher

def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
        
    return inner

In [23]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [24]:
htmlize('1 < 100')

'1 &lt; 100'

In [28]:
# adding some functions to the registry

def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    registry[int] = lambda a: '{0}(<i>{1}</i>)'.format(a, str(hex(a)))
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')
    
    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)
        
    return inner

In [29]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [30]:
htmlize('1 < 100')

'1 &lt; 100'

In [31]:
htmlize(100)

'100(<i>0x64</i>)'

In [57]:
# improving the implementation 

def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
        
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorated.register = register
    decorated.registry = registry
    decorated.dispatch = dispatch
    return decorated

In [58]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [59]:
htmlize('1 < 100')

'1 &lt; 100'

In [60]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

In [62]:
htmlize.registry

{object: <function __main__.htmlize(a)>}

In [64]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

#long form: html_int = htmlize.register(int)(html_int)

In [65]:
htmlize(100)

'100(<i>0x64</i>)'

In [66]:
htmlize.registry

{object: <function __main__.htmlize(a)>, int: <function __main__.html_int(a)>}

In [67]:
# stacking decorators to add same func to multiple keys
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [68]:
htmlize((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [69]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [70]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_sequence(l)>,
 tuple: <function __main__.html_sequence(l)>}

In [71]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [73]:
from numbers import Integral

In [74]:
class Person:
    pass

In [78]:
class Student(Person): # student inherits from Person
    pass

In [79]:
p = Student()

In [80]:
type(p) # student

__main__.Student

In [81]:
isinstance(p, Student)

True

In [82]:
isinstance(p, Person)

True

In [84]:
# built-in version!
from functools import singledispatch

In [85]:
from numbers import Integral
from collections.abc import Sequence

In [86]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [87]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [88]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [89]:
@htmlize.register(Integral)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [90]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.html_integral_number(a)>})

In [91]:
htmlize.dispatch(int)

<function __main__.html_integral_number(a)>

In [92]:
print(type(10), isinstance(10, int), isinstance(10, Integral))

<class 'int'> True True


In [93]:
htmlize.dispatch(bool)

<function __main__.html_integral_number(a)>

In [94]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [95]:
htmlize([1, 2, 3])

'<ul>\n<li>1(<i>0x1</i>)</li>\n<li>2(<i>0x2</i>)</li>\n<li>3(<i>0x3</i>)</li>\n</ul>'

In [97]:
# strings are sequences, so to avoid infinite recursion we need higher specificity type match (str itself)
@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [98]:
htmlize('python')

'python'